    v1 - правки в подсчете ВОЗРАСТА (06-10-23) возраст на дату НАЧАЛА периода

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display_markdown
import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-success">     <font size=3>
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
<b> На основе "мониторинг_2кв_v5_dead_correct" <br>
</b>            </div>

### 
<div class="alert alert-block alert-warning">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> Функции для расчета  <br>
</b>            </div>

#### функция возраста

In [2]:
# -------------AGE -------------
def calculate_age(x,y):
    age = y.year - x.year - ((y.month, y.day) < (x.month, x.day)) 
    if age < 0:
        age = 0
    return age

#### функция 605го

In [3]:
# def load_data_605(file_name, bmchu, bk, date_finish): -- изменения в подсчете ВОЗРАСТА -------------- 06.10.23 ----- возраст считаем для момента ВХОДА в ПЕРИОД (за время рассмотрения дети взрослеют и на конец периода мы их можем не поймать) --------
def load_data_605(file_name, bmchu, bk, date_start):
        
     #     ---------- ЗАГРУЗКА и ПРЕДОБРАБОТКА файла 605го ----------------------
    
    # file_name = file_605
    df = pd.read_csv(file_name, delimiter=';', encoding='cp1251')

    # уберем лишние колонки
    df_605 = df.copy()
    df_605 = df_605[['Дата постановки на учет/включения пациента в реестр',
           'Пациент', 
           'Дата рождения', 'Пол',
           'Дата признания пациента нуждающимся в ПМП (ВК)',
           'Текущий статус пациента',
           'Дата прекращения оказания ПМП', 
             'Курирующее подразделение/филиал',
                   'Гражданский статус' ,
                    'Адрес', 'Основной диагноз по МКБ-10',
                     'Адрес фактический',
                     'Дата разрешения ДЗМ',
                     'Номер разрешения ДЗМ',
                     'БК',
                     'Наблюдение в РЦ'
                    ]]

    #  переименуем покороче столбцы
    df_605.rename(columns={'Дата постановки на учет/включения пациента в реестр': 'Дата_вкл_реестр'}, inplace=True)
    df_605.rename(columns={'Дата рождения': 'ДР'}, inplace=True)
    df_605.rename(columns={'Дата признания пациента нуждающимся в ПМП (ВК)': 'Дата_признания'}, inplace=True)
    df_605.rename(columns={'Дата прекращения оказания ПМП': 'Дата_прекр'}, inplace=True)
    df_605.rename(columns={'Курирующее подразделение/филиал': 'Филиал'}, inplace=True)
    df_605.rename(columns={'Основной диагноз по МКБ-10': 'Диагноз'}, inplace=True)
    df_605.rename(columns={'Гражданский статус': 'Иногород'}, inplace=True)

    df_concat = pd.concat([df_605])


    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()


    # Уберем записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'
    # - удаляем строки. в которых текущий статус не заполнен =  nan
    list_status_ubiraem = ['новый пациент', 'вне реестра', 'врач КЦ', 'Новый пациент', 'Умер без п/о', 'Вне реестра', 'Врач КЦ', 'умер без п/о'] # -----------------записи со СТАТУСОМ  'Умер без п/о', 'Новый пациент', 'Вне реестра', 'Врач КЦ'----------------
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isin(list_status_ubiraem))]   
    df_concat = df_concat.loc[(~df_concat['Текущий статус пациента'].isna())]  # - удаляем строки. в которых текущий статус не заполнен =  nan


    # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bmchu != 'учитываем':
        df_concat = df_concat.loc[df_concat['Филиал'] != 'БМЧУ "Детский хоспис"']  # - удаляем строки. в которых текущий статус не заполнен =  nan
        
        
   #     # ---------------------------------------------------------Уберем 'БМЧУ "Детский хоспис"' из наших данных ----------------------------
    if bk != 'с учетом БК':
        df_concat = df_concat.loc[df_concat['БК'] != 'Да']  # - удаляем строки. в которых текущий статус не заполнен =  nan     

    # УДАЛИМ полные дубликаиы строк (оставляем только ПЕРВЫЕ строки из повторов (из 605го) )
    df_concat = df_concat.drop_duplicates(keep='first')
    df_concat.index.duplicated()
    df_concat.reset_index(drop= True , inplace= True )


    # Заполним поле Дата_первых_сведений данными: 
    #  если Дата_вкл_реестр не пустая, то  Дата_учета = Дата_вкл_реестр

    df_concat.loc[ (~df_concat['Дата_вкл_реестр'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_вкл_реестр']
    df_concat.loc[ (df_concat['Дата_вкл_реестр'].isnull()) & (~df_concat['Дата_признания'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_признания']
    df_concat.loc[ (df_concat['Дата_вкл_реестр'].isnull()) & (df_concat['Дата_признания'].isnull()) & (~df_concat['Дата_прекр'].isnull()) , 'Дата_первых_сведений'] = df_concat['Дата_прекр'] - pd.Timedelta(days = 2)



    # новый признак - Дата_выбытия

    # 1 -- условие ---------------------------
    df_concat.loc[ (  
               ((df_concat['Дата_прекр'].isnull()) & (df_concat['Дата_вкл_реестр'] < '2020-12-31'))  
            &  (  (df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП') )
               )
               , 'Дата_выбытия'] = '01.01.1900'

    # 2 -- условие----------------------------
    df_concat.loc[(~df_concat['Дата_прекр'].isnull()), 'Дата_выбытия'] = df_concat['Дата_прекр']

    # 3 -- условие ---------------------------
    df_concat.loc[
                     (~df_concat['Дата_первых_сведений'].isnull() & df_concat['Дата_выбытия'].isnull() )  
                  &  ((df_concat['Текущий статус пациента'] == 'Умер') | (df_concat['Текущий статус пациента'] == 'Прекращено оказание ПМП')), 'Дата_выбытия'] = df_concat['Дата_первых_сведений'] + pd.Timedelta(days = 2)



    # ----------------форматнем дату к дате
    df_concat['ДР'] = pd.to_datetime(df_concat['ДР'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_вкл_реестр'] = pd.to_datetime(df_concat['Дата_вкл_реестр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_признания'] = pd.to_datetime(df_concat['Дата_признания'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_прекр'] = pd.to_datetime(df_concat['Дата_прекр'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_выбытия'] = pd.to_datetime(df_concat['Дата_выбытия'], dayfirst=True, errors = 'coerce').dt.normalize()
    df_concat['Дата_первых_сведений'] = pd.to_datetime(df_concat['Дата_первых_сведений'], dayfirst=True, errors = 'coerce').dt.normalize()


   #  # ------------подсчитаем ВОЗРАСТ --------------
   # # now = pd.Timestamp('now')
   #  now = pd.Timestamp(date_finish)
   #  df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
   #  df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (now - df_concat['ДР']).astype('<m8[Y]')

   #    ------------------------- ---подсчитаем ВОЗРАСТ -------------- 06.10.23 ----- возраст считаем для момента ВХОДА в ПЕРИОД (за время рассмотрения дети взрослеют и на конец периода мы их можем не поймать) ---------------
    
   # now = pd.Timestamp('now')
    now = pd.Timestamp(date_start)
    # df_concat.loc[(df_concat['Текущий статус пациента'] == 'Умер') & (~df_concat['Дата_выбытия'].isnull()), 'Возраст'] = (df_concat['Дата_выбытия'] - df_concat['ДР']).dt.days//365
    # df_concat.loc[(df_concat['Текущий статус пациента'] != 'Умер'), 'Возраст'] = (pd.Timestamp(date_start) - df_concat['ДР']).astype('<m8[Y]')
    
    df_concat['Возраст'] = np.nan
    # df_concat['Возраст'] = df_concat.apply( lambda x:  calculate_age(x['ДР'], now),axis=1)
    df_concat['Возраст'] = df_concat.apply( lambda x:  calculate_age(x['ДР'], now),axis=1)
   


    #  ------ посчитаем категорию трудоспособный --------
    #  если пол ЖЕНСКИЙ, и возраст Менее 57 -----> Категория - трудоспособный
    #  если пол Мужской, и возраст Менее 62 -----> Категория - трудоспособный

    df_concat['Категория'] = np.nan
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Мужской') & (df_concat['Возраст'] >= 62), 'Категория'] = 'Старше трудосп'
    df_concat.loc[(df_concat['Возраст'] >= 18) & (df_concat['Пол'] == 'Женский') & (df_concat['Возраст'] >= 57), 'Категория'] = 'Старше трудосп'


    #  ------ определим детей ---------------
    df_concat['Дети'] = np.nan
    # df_concat.loc[(df_concat['Возраст'] < 18) & (df_concat['Возраст'] >= 0), 'Дети'] = 'Дети'
    df_concat['Дети'] = df_concat['Возраст'].apply(lambda x: 'Дети' if x <= 17 else np.nan)


    #  ----- Добавим УНИКАЛЬНЫЙ ключ ФИО+ДР - без пробелов ------------

    df_concat['Ключ'] = df_concat['Пациент'].map(str) + (df_concat['ДР'].dt.date).map(str)
    df_concat['Ключ'] = df_concat['Ключ'].str.lower()            # ---- все к нижнему регистру
    df_concat['Ключ'] = df_concat['Ключ'].str.replace(' ', '')   # ---- пробелы долой 


    df_clr = df_concat.loc[ (~df_concat['Дата_первых_сведений'].isnull())]

    # ------ удаление дубликатов ----------
    df_clr = df_clr.drop_duplicates(['Ключ'], keep='first') 
    
    return df_clr  

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Количество пациентов, получателей помощи (чел.)  <br></b>            </div>

In [4]:
# from general ---------------------------------------------  Количество пациентов, получателей помощи (чел.)
def def_data_all_clients(df, start, finish):
    
 # ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------

    df_all = df.loc[ ((df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))

                    ]

    data_all_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)   &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))

                             ]

    data_all_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                             &   (  df['Дети'].isna())
                                    
                                   ]
    
    data_all_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish) 
                             &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                             &   (  df['Категория'] == 'Старше трудосп')    

                                        ]
    
    data_all_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish) 
                         &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start))  
                         &   (  ~df['Дети'].isna()) 

                                  ]
    
    return data_all_clients.shape[0],  data_all_clients_vzros.shape[0], data_all_clients_vzros_trud.shape[0], data_all_clients_deti.shape[0], data_all_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета НОВЫХ пациентов, обратившихся в ЭТОМ периоде (чел.)  <br></b>            </div>

In [5]:
# 4444 ---------------------------------------------  Количество НОВЫХ пациентов, получателей помощи (чел.)
def def_data_new_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------Дата_первых_сведений


    data_new_clients = df.loc[ ((df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish) )

                             ]


    data_new_clients_vzros = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish)   
                             &   (  df['Дети'].isna())
                                    
                                   ]
    
    data_new_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish)   
                             &   (  df['Категория'] == 'Старше трудосп') 

                                        ]
    
    data_new_clients_deti = df.loc[ (df['Дата_первых_сведений'] >= start)  &  (df['Дата_первых_сведений'] < finish) 
                         &   (  ~df['Дети'].isna())    

                                  ]
    
    return data_new_clients.shape[0],  data_new_clients_vzros.shape[0], data_new_clients_vzros_trud.shape[0], data_new_clients_deti.shape[0], data_new_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета УМЕРШИХ пациентов, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [6]:
# 4444 ---------------------------------------------  Количество НОВЫХ пациентов, получателей помощи (чел.)--------------------------------------Дата_первых_сведений
def def_data_dead_clients(df, start, finish):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------


    data_dead_clients = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                              & (df['Текущий статус пациента'] == 'Умер')
                              ]


    data_dead_clients_vzros = df.loc[ (df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                                   &   (  df['Дети'].isna())
                                    & (df['Текущий статус пациента'] == 'Умер')
                                   ]
    
    data_dead_clients_vzros_trud = df.loc[ (df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  ) 
                                 &   (  df['Категория'] == 'Старше трудосп') 
                                  & (df['Текущий статус пациента'] == 'Умер')
                                        ]
    
    data_dead_clients_deti = df.loc[ (df['Дата_первых_сведений'] < finish)      &   ( (df['Дата_выбытия'].isnull())  | ((df['Дата_выбытия'] >= start) &  (df['Дата_выбытия'] < finish))  )
                                   & (  ~df['Дети'].isna())                                     
                                   & (df['Текущий статус пациента'] == 'Умер')
                                  ]
    
    return data_dead_clients.shape[0],  data_dead_clients_vzros.shape[0], data_dead_clients_vzros_trud.shape[0], data_dead_clients_deti.shape[0], data_dead_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета ИНОГОРОДНИХ пациентов, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [11]:
# 100723 ----------------------------------------------ИНОГОРОДНИЕ-----------------------
def def_data_inogorod_clients(df, start, finish, list_):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------  'Гражданин РФ/иногородний', 'Украина', 'БОМЖ', nan, 'Иностранец', 'ЛНР', 'ДНР'

    # list_inogorod = ['гражданин РФ (Иногородний)', 'житель ДНР/ЛНР/новых территорий', 'ДНР','Гражданин РФ/иногородний', 'ЛНР',  'ДНР', 'РФ, ДНР' ]
    
    data_inogorod_clients = df.loc[ ((df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))
                             
                                   & (
                                      (df['Иногород'].isin(list_inogorod)) 
                                       # (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний', 'БОМЖ'])) # -------------------------------------------------------------------- + БОМЖ
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   ) ]


    
    data_inogorod_clients_vzros = df.loc[ ((df['Дата_первых_сведений'] < finish)     &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))  
                                   &   (  df['Дети'].isna())
                                   & (
                                      (df['Иногород'].isin(list_inogorod)) 
                                                                              # (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний', 'БОМЖ'])) # ------------------------------------- + БОМЖ
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    data_inogorod_clients_vzros_trud = df.loc[ ((df['Дата_первых_сведений'] < finish)    &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     ))   
                                 &   (  df['Категория'] == 'Старше трудосп') 
                                  & (
                                      (df['Иногород'].isin(list_inogorod)) 
                                                                             # (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний', 'БОМЖ']))  # ------------------------------------- + БОМЖ
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    data_inogorod_clients_deti = df.loc[ ((df['Дата_первых_сведений'] < finish)      &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start)     )) 
                                   & (  ~df['Дети'].isna())                                     
                                  & (
                                      (df['Иногород'].isin(list_inogorod)) 
                                        # (df['Иногород'].isin(['ЛНР', 'ДНР', 'Гражданин РФ/иногородний', 'БОМЖ']))  # ------------------------------------------------------------------------- + БОМЖ
                                      # | (df['БК'] == 'Да') 
                                      # | ((df['Иногород'] == 'Гражданин РФ/иногородний') & ((~df['Дата разрешения ДЗМ'].isna())) | (~df['Номер разрешения ДЗМ'].isna()) )
                                   )]
    
    return data_inogorod_clients.shape[0],  data_inogorod_clients_vzros.shape[0], data_inogorod_clients_vzros_trud.shape[0], data_inogorod_clients_deti.shape[0], data_inogorod_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Иностарцев и БОМЖей, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [22]:
# 100723 ----------------------------------------------Иностарцев-----------------------
def def_data_inostran_clients(df, start, finish, list_):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------  'Гражданин РФ/иногородний', 'Украина', 'БОМЖ', nan, 'Иностранец', 'ЛНР', 'ДНР'

    # list_inostran = ['Иностранец', 'иностранец',  ]
    # list_bomj = ['лицо без определенного места жительства', 'БОМЖ' ]
    
    all_clients_period = df.loc[ 
                                   (df['Дата_первых_сведений'] < finish)    
                               &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start) ) ]

    data_inostran_clients            = all_clients_period[all_clients_period['Иногород'].isin(list_inostran + list_bomj)]
    
    data_inostran_clients_vzros      = data_inostran_clients[data_inostran_clients['Дети'].isna()]
    
    data_inogorod_clients_vzros_trud = data_inostran_clients_vzros[data_inostran_clients_vzros['Категория'] == 'Старше трудосп']
    
    data_inostran_clients_deti       = data_inostran_clients[~data_inostran_clients['Дети'].isna()]
    
    
    return data_inostran_clients.shape[0],  data_inostran_clients_vzros.shape[0], data_inogorod_clients_vzros_trud.shape[0], data_inostran_clients_deti.shape[0], data_inostran_clients

### 
<div class="alert alert-block alert-info">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=3>
<b> функция подсчета Укр-ЛНР - ДНР, в ЭТОМ периоде (чел.)  <br></b>            </div>

In [13]:
# 100723 ----------------------------------------------Иностарцев-----------------------
def def_data_ukr_dnr_clients(df, start, finish, list_):
    
#  ------------------- берем     Дата_первых_сведений ----------------- если брать дату включения - будет меньше ---------  'Гражданин РФ/иногородний', 'Украина', 'БОМЖ', nan, 'Иностранец', 'ЛНР', 'ДНР'

    # list_ukr_dnr = ['житель ДНР/ЛНР/новых территорий', 'граждане Украины', 'ДНР', 'ЛНР', 'Украина', 'ДНР' 'РФ, ДНР'  ]
   
    all_clients_period = df.loc[ 
                                   (df['Дата_первых_сведений'] < finish)    
                               &   ( (df['Дата_выбытия'].isnull())  | (df['Дата_выбытия'] >= start) ) ]

    data_ukr_dnr_clients = all_clients_period[all_clients_period['Иногород'].isin(list_ukr_dnr)]
    
    data_ukr_dnr_clients_vzros = data_ukr_dnr_clients[data_ukr_dnr_clients['Дети'].isna()]
    
    data_ukr_dnr_clients_vzros_trud = data_ukr_dnr_clients_vzros[data_ukr_dnr_clients_vzros['Категория'] == 'Старше трудосп']
    
    data_ukr_dnr_clients_deti = data_ukr_dnr_clients[~data_ukr_dnr_clients['Дети'].isna()]
    
    
    return data_ukr_dnr_clients.shape[0],  data_ukr_dnr_clients_vzros.shape[0], data_ukr_dnr_clients_vzros_trud.shape[0], data_ukr_dnr_clients_deti.shape[0], data_ukr_dnr_clients

### 
<div class="alert alert-block alert-success">
<!-- <div class="alert alert-block alert-info"> <div class="alert alert-block alert-warning"> <div class="alert alert-block alert-success"> <div class="alert alert-block alert-danger">      --> 
    <font size=6>
<b> РЕЗУЛЬТАТ Статистика ОБЩИЕ ПОКАЗАТЕЛИ <br></b>            </div>

'2023-01-01_2023-12-31'

In [28]:
# ---------------------------- ИТОГИ ---------------
start = '2023-01-01'
finish = '2023-12-31'
# file_605 = 'выгрузка_605_из_1с_0610.csv'
file_605 = '1С_выгрузка_из_1с_20240110_0832.csv'


bmchu = 'не учитываем'
# bmchu = 'учитываем'
bk = 'с учетом БК'
# bk = 'без учета БК'


god_start = start.split('-')[0]
mes_start = start.split('-')[1]
day_start = start.split('-')[2]

god_finish = finish.split('-')[0]
mes_finish = finish.split('-')[1]
day_finish = finish.split('-')[2]



list_inogorod = ['гражданин РФ (Иногородний)', 'житель ДНР/ЛНР/новых территорий', 'ДНР','Гражданин РФ/иногородний', 'ЛНР',  'ДНР', 'РФ, ДНР' ]
list_inostran = ['Иностранец', 'иностранец',  ]
list_bomj = ['лицо без определенного места жительства', 'БОМЖ' ]
list_ukr_dnr = ['житель ДНР/ЛНР/новых территорий', 'граждане Украины', 'ДНР', 'ЛНР', 'Украина', 'ДНР' 'РФ, ДНР'  ]

# ------------------------------------------------готовим реестр ------
df_reest = load_data_605(file_605, bmchu, bk,start)   # ---- start в связи с уточнениями по вычислению возраста

# ------------------------------------------------собираем данные ------
all_clients  = def_data_all_clients(df_reest, start, finish)
new_clients  = def_data_new_clients(df_reest, start, finish)
dead_clients = def_data_dead_clients(df_reest, start, finish)
inogorod_clients = def_data_inogorod_clients(df_reest, start, finish, list_inogorod)
inostr_clients = def_data_inostran_clients(df_reest, start, finish, (list_inostran + list_bomj))
ukr_clients = def_data_ukr_dnr_clients(df_reest, start, finish, list_ukr_dnr)

a = f'Статистика МОНИТОРИНГ период с {day_start}.{mes_start}.{god_start} по {day_finish}.{mes_finish}.{god_finish}:'

display_markdown(f''' # {a}

| Параметр | Количество всего| Взрослые |трудоспособ | Дети |Условия отбора|файл|
|---:|:-|:-|:-|:-|--|--|
| Количество пациентов __ВСЕГО__ за период __БМЧУ__{bmchu}__ и __{bk}__| {all_clients[0]}      | {all_clients[1]} |{all_clients[2]} |{all_clients[3]} |--|all_clients.xlsx|
| Количество __НОВЫХ__ пациентов за период          | {new_clients[0]}      | {new_clients[1]} |{new_clients[2]} |{new_clients[3]} |--|--|
| Количество __УМЕРШИХ__ пациентов за период        | {dead_clients[0]}     | {dead_clients[1]} |{dead_clients[2]} |{dead_clients[3]} |--|--|
| Количество __ИНОГОРОДНИХ__ __{bk}__ пациентов за период    | {inogorod_clients[0]} | {inogorod_clients[1]} |{inogorod_clients[2]} |{inogorod_clients[3]} |__{list_inogorod[:]}__|df_inogorod_clients.xlsx|
| Количество __ИНОСТРАННЫХ__ пациентов __{bk}__ за период    | {inostr_clients[0]} | {inostr_clients[1]} |{inostr_clients[2]} |{inostr_clients[3]} |__{list_inostran + list_bomj}__|df_inostr_clients.xlsx|
| Количество пациентов ___Укр+нов.терр__ __{bk}__ за период    | {ukr_clients[0]} | {ukr_clients[1]} |{ukr_clients[2]} |{ukr_clients[3]} |__{list_ukr_dnr}__|df_inostr_clients.xlsx|


''', raw=True)

# ----- сохраним данные в файлы ---------
all_clients[4].to_excel(f'all_clients_{start + "_" + finish}.xlsx')
inogorod_clients[4].to_excel(f'df_inogorod_clients_{start + "_" + finish}.xlsx')
inostr_clients[4].to_excel(f'df_inostr_clients_{start + "_" + finish}.xlsx')
ukr_clients[4].to_excel(f'df_ukr_clients_{start + "_" + finish}.xlsx')

 # Статистика МОНИТОРИНГ период с 01.01.2023 по 31.12.2023:

| Параметр | Количество всего| Взрослые |трудоспособ | Дети |Условия отбора|файл|
|---:|:-|:-|:-|:-|--|--|
| Количество пациентов __ВСЕГО__ за период __БМЧУ__не учитываем__ и __с учетом БК__| 29897      | 29257 |24861 |640 |--|all_clients.xlsx|
| Количество __НОВЫХ__ пациентов за период          | 18899      | 18705 |16011 |194 |--|--|
| Количество __УМЕРШИХ__ пациентов за период        | 15772     | 15697 |13583 |75 |--|--|
| Количество __ИНОГОРОДНИХ__ __с учетом БК__ пациентов за период    | 207 | 189 |121 |18 |__['гражданин РФ (Иногородний)', 'житель ДНР/ЛНР/новых территорий', 'ДНР', 'Гражданин РФ/иногородний', 'ЛНР', 'ДНР', 'РФ, ДНР']__|df_inogorod_clients.xlsx|
| Количество __ИНОСТРАННЫХ__ пациентов __с учетом БК__ за период    | 167 | 151 |79 |16 |__['Иностранец', 'иностранец', 'лицо без определенного места жительства', 'БОМЖ']__|df_inostr_clients.xlsx|
| Количество пациентов ___Укр+нов.терр__ __с учетом БК__ за период    | 98 | 95 |64 |3 |__['житель ДНР/ЛНР/новых территорий', 'граждане Украины', 'ДНР', 'ЛНР', 'Украина', 'ДНРРФ, ДНР']__|df_inostr_clients.xlsx|




In [ ]:
# all_clients[4].to_excel('all_clients_25266.xlsx')